<center><b><font size=6>Machine Learning for Networks<b><center>

<left><b><font size=4>SSH Shell Attack session<b><left>                                                                   
##### Group 11

In [ ]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.dates as mdates
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score
import time

# Models 
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import multilabel_confusion_matrix, classification_report

#Language Models Exploration 
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense  

# Clustering
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, adjusted_rand_score, rand_score

<br>
<left><b><font size=4>Section 1 – Data exploration and pre-processing<b><left>

Dataset with all the SSH sessions

In [ ]:
# Loading the dataset
df_original = pd.read_parquet('ssh_attacks.parquet')
df=df_original.copy()
# Convert 'first_timestamp' column to datetime objects
df['first_timestamp'] = pd.to_datetime(df['first_timestamp'])
df

**1. When are the attacks performed? Analyze the temporal series.**

In [ ]:
# Creating a new DataFrame containing only attack instances
# Excluding sessions labeled as "Harmless" with a single label
df_attacks = df.loc[~df["Set_Fingerprint"].apply(lambda x : "Harmless" in x and len(x) == 1)]

# Count the number of attacks per day
# Extract the date from 'first_timestamp', count occurrences, sort by date
attacks_per_day = df_attacks['first_timestamp'].dt.date.value_counts().sort_index().to_frame("Number_of_attacks_per_day")

# Display the resulting DataFrame showing the number of attacks per day
attacks_per_day

The analysis of attack occurrences based on the temporal series of first_timestamp showcases a trend in attack frequency over time.
The dataset has been transformed to interpret the first_timestamp column as datetime objects for accurate temporal analysis. The subsequent process isolates attack instances within the dataset, excluding records tagged as "Harmless" with a single label.
The resulting analysis presents the number of attacks per day:

| Date_time | Attacks |
| --- | --- |
|June 4th, 2019 | 82 attacks |
|June 5th, 2019 | 124 attacks |
|June 6th, 2019 | 117 attacks |
|June 7th, 2019 | 121 attacks |
|June 8th, 2019 | 118 attacks |
| ... (continues with dates up to) |
| February 25th, 2020| 649 attacks |
| February 26th, 2020| 483 attacks |
| February 27th, 2020| 551 attacks |
| February 28th, 2020| 580 attacks |
| February 29th, 2020| 627 attacks |

This temporal series reveals fluctuations in attack intensity over time, with notable spikes and drops in attack occurrences. The observations suggest potential patterns or trends that could be further explored to understand the dynamics of these SSH shell attacks across different periods.

In [ ]:
# Set the figure size for the plot
plt.figure(figsize=(8,5))
sns.set_style("darkgrid")

# Create a line plot using seaborn
sns.lineplot(
    data = attacks_per_day,          # Use the 'attacks_per_day' DataFrame for plotting
    x = attacks_per_day.index,       # X-axis represents the timestamp of attacks
    y = "Number_of_attacks_per_day", # Y-axis represents the number of attacks per day
    marker='o',                      # Marker style for data points
    linestyle='-',                   # Style of the line connecting the data points
    color= 'blue',                   # Color of the line
    markersize=5                     # Size of markers
)

# Set plot title and labels for axes
plt.title('Number of Attacks')
plt.xlabel("Date")
plt.ylabel("Number of attacks")

# Format the date on the x-axis to display day-month-year
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d-%m-%y'))
plt.xticks(rotation=45, ha='right')

# Ensure proper layout and display the plot
plt.tight_layout()
plt.show()

The visualization depicting attack frequencies over time reveals distinct patterns:

Between June 2019 and September 2019, there is a conspicuous decrease in attack occurrences, indicating a phase of minimal activity. Subsequently, there is a significant surge in attack instances towards the latter part of 2019, signifying a notable rise in both the frequency and intensity of attacks during this period.

This timeline underscores a stark contrast between the relatively quiet phase observed from June to September 2019 and the pronounced escalation in attack activities, particularly notable in the latter months of the year. This shift in trend emphasizes a substantial alteration in attack behavior, marked by an extended period of low activity succeeded by a considerable surge in attack incidents towards the year's end.

Moreover, at the beginning of 2020, there is a noticeable decline in attack occurrences once more. This decline follows the heightened activity observed in late 2019, representing a shift from the increased attack rates back to a decreased frequency as the year transitions into its initial months.

In [ ]:
#This code aggregates attack instances per year, creating a DataFrame named attacks_per_year

# Group attack instances by year and count occurrences
attacks_per_year = df_attacks.groupby(df['first_timestamp'].dt.year).size().to_frame("Number_attacks").reset_index()

# Rename columns for clarity
attacks_per_year.rename(columns={"first_timestamp": "Year"}, inplace=True)
attacks_per_year

The contrast in attack counts between the two years, with 2019 showing a significantly higher number of attacks compared to 2020, can be reasonably attributed to the limited temporal coverage of the dataset for the year 2020. With data available for only two months of 2020, the reduced number of observations in this period is expected and explains the lower count of attacks for that year compared to the extensive records available for 2019.

In [ ]:
# Set the figure size for the plot
plt.figure(figsize=(7,5))

# Set the figure size for the plot
sns.barplot(
    data = attacks_per_year,   # Use the 'attacks_per_year' DataFrame for plotting
    x = "Year",                # X-axis represents the years (2019 and 2020)
    y = "Number_attacks",      # Y-axis represents the number of attacks
    hue= "Year",
    palette = ['blue','orange']
)

# Set plot labels and title
plt.ylabel("Number of attacks")
plt.title("Number of total attacks in 2019 and 2020")

# Display the plot
plt.tight_layout()
plt.show()

So to study the trend of the number of attacks during the time range considered, would be usefull to analyse data an year at time, as the the chart below is showing: 

In [ ]:
# Filter attack instances for the year 2019
df_attacks_2019 = df_attacks.loc[df_attacks["first_timestamp"].dt.year == 2019]
# Group attacks by month and count occurrences
df_attacks_2019_month = df_attacks_2019.groupby(df_attacks_2019["first_timestamp"].dt.month).size().to_frame("Number_attacks_2019_month").reset_index()

# Filter attack instances for the year 2020
df_attacks_2020 = df_attacks.loc[df_attacks["first_timestamp"].dt.year == 2020]
# Group attacks by month and count occurrences
df_attacks_2020_month = df_attacks_2020.groupby(df_attacks_2020["first_timestamp"].dt.month).size().to_frame("Number_attacks_2020_month").reset_index()

In [ ]:
# Set up a figure with two subplots
plt.figure(figsize=(10,5))
sns.set_style("darkgrid")

# Plot for 2019
plt.subplot(1,2,1)
sns.barplot(
    data = df_attacks_2019_month,
    x = "first_timestamp",
    y = "Number_attacks_2019_month",
    color = 'blue'  
)
plt.xlabel("Month")
plt.ylabel("Number of attacks")
plt.title("2019")

# Plot for 2020
plt.subplot(1,2,2)
sns.barplot(
    data = df_attacks_2020_month,
    x = "first_timestamp",
    y = "Number_attacks_2020_month",
    color = 'orange'
)
plt.xlabel("Month")
plt.ylabel("Number of attacks")
plt.title("2020")

# Adjust layout and display the plot
plt.tight_layout(pad = 5)
plt.show()

The bar charts reveal intriguing trends:

- In 2019, there is a noticeable surge in the number of attacks from months 9 to 12. A progressive increase in attack activity is observed during these months, reaching a peak towards the year's end.

- At the onset of 2020, in months 1 and 2, a relatively similar frequency of attacks is noted, approximately representing half the number of attacks compared to month 10 in 2019.

These patterns outline a significant uptick in attack activity towards the end of 2019, followed by a comparatively steady beginning in 2020 with a considerably lower number of attacks compared to the peak period of the previous year.


Until now we worked to the whole dataset of sessions. But there are a part of them that bring to harmless attaks. So sessions that are been analysed by the honeypot that was collecting the shell attacks, but finally they're been classified as harmless.

Let's have a look to the temporal distribution of these harmless sessiosns:

In [ ]:
# Filtering harmless sessions based on 'Set_Fingerprint' column
df_harmless = df[df["Set_Fingerprint"].apply(lambda x : "Harmless" in x and len(x) == 1)]

# Counting harmless sessions per day
harmless_per_day = df_harmless['first_timestamp'].dt.date.value_counts().sort_index().to_frame("Number_harmless_per_day")

# Creating a bar plot
plt.figure(figsize=(12, 6))
sns.set_style("darkgrid")
sns.barplot(
    data = harmless_per_day,
    x = harmless_per_day.index, 
    y = "Number_harmless_per_day",
    hue = harmless_per_day.index,
    legend = False,
    palette = "icefire"
)
plt.title('Temporal Distribution of Harmless Sessions')
plt.xlabel('Date')
plt.ylabel('Number of Harmless Sessions')
plt.xticks(rotation = 45, fontsize = 4)  # Rotate x-axis labels by 45 degrees

plt.show()

So we can't see a particular distribution, or periods with important differences in terms of the number of harmless attacks.
These harmless attacks can be errors in the scanning tools or false positive.  

**1.2. Extract features from the attack sessions. How does the empirical distribution of the number of
characters in each session look like? How is the distribution of the number of word per session?**

In [ ]:
def clean_session(full_session):
    return [word for word in list(filter(None, re.split(";|/|-|\||\.|=|$| ", full_session))) if word.isalpha()]

In [ ]:
# Create another dataframe to show the number of characters and words in each session
# Number_characters: Calculates the number of alphanumeric characters in each session.
number_characters = df['full_session'].apply(lambda x: len([char for char in x if char.isalpha()]))

# Number_words: Computes the number of words (considering only alphanumeric characters) in each session.
# the regex splits the bash command with multiple delimiters (;, /, -, |,  ,)
number_words = df['full_session'].apply(lambda x: len(clean_session(x)))

data = {"number_characters": number_characters, "number_words": number_words}
df_number_characters_words = pd.DataFrame(data = data)
df_number_characters_words

In [ ]:
# This code creates two side-by-side histograms:
# Histogram of Character Counts per Session: Displays the distribution of the number of characters in each session. It bins the data into 50 bins.
# Histogram of Word Counts per Session: Shows the distribution of the number of words in each session. It also bins the data into 50 bins.
# These histograms help visualize the distributions of character and word counts within the attack sessions, providing insights into the length and complexity of these sessions in terms of characters and words. 

# Tracking histograms for character and word counts per session.
plt.figure(figsize=(11, 5))

# Histogram for the number of characters per session.
plt.subplot(1, 2, 1)
sns.set_style("darkgrid")
plt.hist(df_number_characters_words['number_characters'], bins=50, color = 'blue')
plt.title('Distribution of the number of characters per session')
plt.xlabel('Number of Characters')
plt.ylabel('Frequency')


# Histogram for the number of words per session
plt.subplot(1, 2, 2)
plt.hist(df_number_characters_words['number_words'], bins = 50 , color = 'orange') 
plt.title('Distribution of the number of words per session')
plt.xlabel('Number of Words')
plt.ylabel('Frequency')


plt.tight_layout()
plt.show()

From the plot, it's possible to see that the distribution of words per session and the distribution of characters per session predominantly concentrate below 2000 words and below 20,000 characters, respectively.

To have a clearer idea of the distribution, we decided to limit the x-axis for both plots so that we can closely examine these two distributions

In [ ]:
# Tracking histograms for character and word counts per session.
plt.figure(figsize=(11, 5))

# Histogram for the number of characters per session.
plt.subplot(1, 2, 1)
sns.set_style("darkgrid")
plt.hist(df_number_characters_words['number_characters'], bins=50, range=(0, 700), color = 'blue')
plt.title('Distribution of the number of characters per session')
plt.xlabel('Number of Characters')
plt.ylabel('Frequency')
plt.xlim(0, 700)

# Histogram for the number of words per session
plt.subplot(1, 2, 2)
plt.hist(df_number_characters_words['number_words'], bins = 50, range = (0, 70) , color = 'orange')
plt.title('Distribution of the number of words per session')
plt.xlabel('Number of Words')
plt.ylabel('Frequency')
plt.xlim(0, 70)

plt.tight_layout()
plt.show()

From the histograms, it is observed that the distribution of character counts per session centers is concentrated around approximately 350 characters.
Moreover, regarding the number of words per session, the distribution frequently peaks around 40 to 48 words. This indicates that sessions often contain this range of word counts, emphasizing a typical occurrence of sessions with this word count range. These insights provide a clear understanding of the common lengths observed within the attack sessions, both in terms of characters and words.

**1.3. What are the most common words in the sessions?**

In [ ]:

# Joining all text from 'full_session' into a single string
full_session = ' '.join(df['full_session'])

# Cleaning the text, leaving only alpha numeric words
session_cleaned = clean_session(full_session)

In [ ]:
# Count the occurrences of each word and extract the top 10 most common words
df_list_session_words = pd.Series(session_cleaned).value_counts().head(10)

# Identify the most common word and its frequency
most_common_word = df_list_session_words.idxmax()    # Most common word
frequency = df_list_session_words.max()              # Frequency of the most common word

# Convert the Series of word frequencies to a dictionary
word_freq=df_list_session_words.to_dict()
most_common_word

The most common word is : 'tmp'

In [ ]:
# dictionary that holds the frequencies of the top 10 most common words
word_freq

In [ ]:
# Create a WordCloud object
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_freq)

# Display the WordCloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

 This representation emphasizes words based on their frequency in the sessions. Words that appear more frequently will be displayed larger and more prominently within the WordCloud. The interpolation='bilinear' argument enhances the image quality for better clarity. The plt.axis('off') command removes the axis for a cleaner visual appearance.

We find that many common linux command are used in the attack sessions. 
- "**grep**", the most frequent of the set, can be useful for instance to find some text pattern in files, or to find some specific file/directory in the system. 

- "**Cat**" is used to display content of file, that's what an attacker shoud want, to look at some sensitive data, private data or configuration files for example.

- "**Rm**" could indicate an attempt to remove files/directories. 

And so on...


**1.4. How are the intents distributed? How many intents per session do you observe? What are the most common intents? How are the intents distributed in time?**

In [ ]:
# Extracting relevant columns: session_id and Set_Fingerprint
intents_df = df[["session_id", "Set_Fingerprint"]]

# Expanding the 'Set_Fingerprint' column to individual intents and sessions
intents_df_exploted = intents_df.explode('Set_Fingerprint')

# Grouping by session and counting the number of intents per session
intents_df_grouped = intents_df_exploted.groupby("session_id").size()

# Plotting the distribution of intents per session

#TODO SHOULD WE REMOVE THIS PLOT ?

plt.figure(figsize=(10,8))
sns.kdeplot(
    data = intents_df_grouped
)
plt.xlabel("Sessions")
plt.ylabel("Number of intents")
plt.title("Number of Intents per Sessions")
plt.show()

We can see that many sessions have differents intents. Obviously an attacker, once inside the server, probably wants to damage the system in differents ways.

In [ ]:
intents = intents_df_exploted.groupby('Set_Fingerprint').count().sort_values(by='session_id', ascending=False)

# Distribution plot of intents 
# Set up Seaborn style
sns.set(style="darkgrid")

# Generate the figures
plt.figure(figsize=(6,3))
sns.barplot(intents, x='session_id', y=intents.index, color='blue', hue_order=intents.index)
plt.title('Distribution of Intents')
plt.xlabel('Number of intents')
plt.ylabel('Type of Intents')

# Show the plot
plt.tight_layout()
plt.show()

The presented bar chart shows the most common intentions found in the dataset; Discovery, Persistence and Execution lead the most used type of attacks for each of the sessions.

In [ ]:
#TODO REVIEW THIS PLOT

# Explode the 'Set_Fingerprint' column and group by timestamp and intents, filling missing values with 0
df_grouped = df.explode('Set_Fingerprint').groupby([pd.Grouper(key='first_timestamp', freq='D'), 'Set_Fingerprint']).size().unstack().fillna(0)

# Plotting the distribution of intents over time
df_grouped.plot(
    figsize = (10,6), 
    xlabel = "Date", 
    ylabel = "Distribution of the Intents", 
    title = "Distribution of the Intents in Time")
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%d-%m-%y'))

The distribution of intents unfolds as follows:

**Defense Evasion**:
No significant peaks noticed, with a few attacks observed between July and September 2019, remaining consistently low alongside 'Harmless', 'Impact', and 'Other' intents.

**Execution**:
Displays a sharp peak towards the end of 2019, notably in the last two months.

**Persistence and Discovery**:
Showcase an intriguing trend, reaching their highest peaks towards the end of 2019. These intents exhibit the highest frequency, notably surging towards the year-end, reaching maximum levels.

**1.5. How can text represented numerically? Try to convert the text into numerical representations
(vectors) through Bag of Words (BoW)**

In [ ]:
df_session_cleaned = df.copy()
df_session_cleaned.update(df_session_cleaned["full_session"].apply(lambda x : str(clean_session(x))))
df_session_cleaned

In [ ]:
count_vectorizer = CountVectorizer(min_df = 0.05)
bow = count_vectorizer.fit_transform(df_session_cleaned["full_session"])

In [ ]:
df_session_cleaned_bow = pd.DataFrame(bow.toarray(), index=df_session_cleaned.index, columns = list(count_vectorizer.vocabulary_.keys()))
df_session_cleaned_bow = pd.concat([df_session_cleaned, df_session_cleaned_bow], axis=1)
df_session_cleaned_bow

**1.6. Associate each word in each attack session with its TF-IDF value (Term Frequency-Inverse Document Frequency)**

TF-IDF (Term Frequency-Inverse Document Frequency) is a method to evaluate the importance of words in a document. 

It can do it measuring how often a word appears in a document and, if we have many documents, the frequency along all the documents.

In [ ]:
tfidf_vectorizer = TfidfVectorizer(min_df = 0.05)
tfid = tfidf_vectorizer.fit_transform(df_session_cleaned["full_session"])

In [ ]:
df_session_cleaned_tfidf = pd.DataFrame(tfid.toarray(), index=df_session_cleaned.index, columns = list(tfidf_vectorizer.vocabulary_.keys()))
df_session_cleaned_tfidf = pd.concat([df_session_cleaned, df_session_cleaned_tfidf], axis=1)
df_session_cleaned_tfidf

Once applied TF-IDF, we have a new dataframe with 61 new features corresponding to the differents words used in the session. 
But 61 is to much for us, will require an heavy computational power, and so much time to train the models.
So we want to reduce the number fo features. To do that we firstly compute the correlation matrix of only the 61 features.

<br>
<b>Correlation Matrix</b>

In [ ]:
# Compute the correlation matrix
df_features_bow = df_session_cleaned_bow.drop(columns=["session_id", "full_session", "first_timestamp", "Set_Fingerprint"])
correlation_matrix_bow = df_features_bow.corr().abs()

# Compute the heatmap of the correlation matrix}
plt.figure(figsize=(50,50))
sns.heatmap(correlation_matrix_bow, cmap='Blues', annot=True, vmin=.0, vmax=1, cbar_kws={'label':'Correlation'})
plt.title('Correlation matrix')
plt.show()

In [ ]:
# Extract features having a correlation (with a covariance) > 0.98
c = correlation_matrix_bow[correlation_matrix_bow > 0.8]
s = c.unstack()
so = s.sort_values(ascending=False).reset_index()

# Get strongly correlatead features, removing pairs having correlation = 1 because of the diagonal,
# i.e., correlation between one feature and itself"
so = so[(so[0].isnull()==False) & (so["level_0"] != so["level_1"])]
to_be_deleted = []
candidates = list(so["level_0"])

# Get the unique set of features to be deleted
# Notice that we discard one feature per time considering the case where a feature is strongly correlated
# with multiple features
subset_so = so
for candidate in candidates:
    if (candidate in list(subset_so["level_0"])): 
        to_be_deleted.append(candidate) # add the feature to the removed candidates
        # remove the rows where the removed feature is involved
        subset_so = subset_so[(subset_so["level_0"] != candidate) & (subset_so["level_1"] != candidate)] 
print(len(to_be_deleted), 'features to be removed')
to_be_deleted

In [ ]:
# Compute the correlation matrix
df_features_tfidf = df_session_cleaned_tfidf.drop(columns=["session_id", "full_session", "first_timestamp", "Set_Fingerprint"])
correlation_matrix_tfidf = df_features_tfidf.corr().abs()

# Compute the heatmap of the correlation matrix}
plt.figure(figsize=(50,50))
sns.heatmap(correlation_matrix_tfidf, cmap='Blues', annot=True, vmin=.0, vmax=1, cbar_kws={'label':'Correlation'})
plt.title('Correlation matrix')
plt.show()

In [ ]:
# Extract features having a correlation (with a covariance) > 0.98
c = correlation_matrix_tfidf[correlation_matrix_tfidf > 0.8]
s = c.unstack()
so = s.sort_values(ascending=False).reset_index()

# Get strongly correlatead features, removing pairs having correlation = 1 because of the diagonal,
# i.e., correlation between one feature and itself"
so = so[(so[0].isnull()==False) & (so["level_0"] != so["level_1"])]
to_be_deleted = []
candidates = list(so["level_0"])

# Get the unique set of features to be deleted
# Notice that we discard one feature per time considering the case where a feature is strongly correlated
# with multiple features
subset_so = so
for candidate in candidates:
    if (candidate in list(subset_so["level_0"])): 
        to_be_deleted.append(candidate) # add the feature to the removed candidates
        # remove the rows where the removed feature is involved
        subset_so = subset_so[(subset_so["level_0"] != candidate) & (subset_so["level_1"] != candidate)] 
print(len(to_be_deleted), 'features to be removed')
to_be_deleted

In [ ]:
from sklearn.decomposition import PCA
pca_bow = PCA()
pca_bow.fit(df_features_bow)
explained_variance_bow = pca_bow.explained_variance_ratio_
cumul_exp_var_bow = np.cumsum(explained_variance_bow)
perc_cumul_exp_var_bow = cumul_exp_var_bow * 100

circle = plt.Circle((10, 99.9), 0.5, color="red", fill=False)

plt.figure(figsize=(10, 8))
ax = sns.lineplot(
    data = perc_cumul_exp_var_bow,
    marker="o"
)
ax.set_aspect("equal")
ax.add_patch(circle)
plt.xlabel("Principal Components")
plt.ylabel("Cumulative explained variance [%]")
plt.grid()
plt.title("PCA")
plt.tight_layout()
plt.show()

In [ ]:
pca_bow = PCA(n_components=10)
pca_result_bow = pca_bow.fit_transform(df_features_bow)
pca_result_bow = pd.DataFrame(pca_result_bow, columns=[f'PC{i}' for i in range(pca_bow.n_components_)])
pca_result_bow

In [ ]:
correlation_matrix_pca_bow = pca_result_bow.corr().abs()
plt.figure(figsize=(10,10))
sns.heatmap(correlation_matrix_pca_bow, cmap='Blues', annot=True, vmin=.0, vmax=1, cbar_kws={'label':'Correlation'}, fmt='.2f')
plt.xlabel('Feature')
plt.ylabel('Feature')
plt.title('Correlation matrix')
plt.show()

In [ ]:
pca_tfidf = PCA()
pca_tfidf.fit(df_features_tfidf)
explained_variance_tfidf = pca_tfidf.explained_variance_ratio_
cumul_exp_var_tfidf = np.cumsum(explained_variance_tfidf)
perc_cumul_exp_var_tfidf = cumul_exp_var_tfidf * 100

circle = plt.Circle((20,99.7), 0.5, color="red", fill=False)

plt.figure(figsize=(10, 8))
ax = sns.lineplot(
    data = perc_cumul_exp_var_tfidf,
    marker="o"
)
ax.set_aspect("equal")
ax.add_patch(circle)
plt.xlabel("Principal Components")
plt.ylabel("Cumulative explained variance [%]")
plt.grid()
plt.title("PCA")
plt.tight_layout()
plt.show()

In [ ]:
pca_tfidf = PCA(n_components=20)
pca_result_tfidf = pca_tfidf.fit_transform(df_features_tfidf)
pca_result_tfidf = pd.DataFrame(pca_result_tfidf, columns=[f'PC{i}' for i in range(pca_tfidf.n_components_)])
pca_result_tfidf

In [ ]:
correlation_matrix_pca_tfidf = pca_result_tfidf.corr().abs()
plt.figure(figsize=(10,10))
sns.heatmap(correlation_matrix_pca_tfidf, cmap='Blues', annot=True, vmin=.0, vmax=1, cbar_kws={'label':'Correlation'}, fmt='.2f')
plt.xlabel('Feature')
plt.ylabel('Feature')
plt.title('Correlation matrix')
plt.show()

In [ ]:
#NOTE I STOPPED HERE TO REVIEW THE PROJECT

In [ ]:
df_features = df_features.drop(columns=to_be_deleted)

In [ ]:
df_features

In [ ]:
print('The final DataFrame has :', df_features.shape[1], 'features.')

In [ ]:
result_df_corr = pd.concat([df_session_cleaned_bow[["session_id", "full_session","first_timestamp","Set_Fingerprint"]], df_features], axis=1)
result_df_corr

<br>
<left><b><font size=4>Section 2 – Supervised Learning – Classification<b><left>

<br>
<div style="text-align: justify"> Classify the tactics of an attack session, based on the used words in the text and also possibly on time. Notice that each session have multiple labels. Hence you can decompose the problem into multiple binary classification problems. For each attack session, you have to solve the 7 binary classification problem, one for each possible label {'Persistence', 'Discovery', 'Defense Evasion', 'Execution', 'Impact', 'Other', 'Harmless'}. </div><br>

**2.1 Perform a split to segment the dataset into training and test dataset. If you want to standardize your dataset, fit the scaler on training set and transforming both training and test. Notice that the sklearn implementation of tf-idf already performs the standardization.**

In [ ]:
features_names = df_features.columns

X_feature = result_df_corr.filter(features_names)
y_feature =  result_df_corr["Set_Fingerprint"]

X_train, X_test, y_train, y_test = train_test_split(
    X_feature,
    y_feature,
    train_size = 0.7,             # 70% of the data is for trainning
)

In [ ]:
print('The size of trainning set is:', len(X_train))
print('The size of test set is:', len(X_test))

In [ ]:
X_train

### Data Preprocessing

##### Standardization of the Numerical Features

As the TF-IDF pre-processing was applied previously to all the sessions, the data considered as features was already standardized. 

##### Standardization of the Categorical Labels 

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# Use MultiLabelBinarizer to transform the labels
mlb = MultiLabelBinarizer()
y_train_mlb = mlb.fit_transform(y_train)
y_test_mlb = mlb.transform(y_test)

**Used Preprocessing Techniques** 
<br>
<div style="text-align: justify"> A <b>MultiLabelBinarizer</b> is a transformer that is used for multi-label classification problems, in order to handle the cases where each sample belongs to multiple classes simultaneously. The purpose of MultiLabelBinarizer is to convert a collection of sequences of labels into a binary matrix format. The binary classification of each label in the 'Set_Fingerprint' column was performed by converting the multi-class label matrix into a binary matrix, where each column represents one of the possible classes and each row represents one instance. </div><br>

<div style="text-align: justify"> <b>TF-IDF </b> The TF-IDF value helps to identify words that are both frequent within a document and distinctive across the entire corpus. This dual consideration allows TF-IDF to highlight terms that are potentially more meaningful for distinguishing documents in tasks such as text classification, information retrieval, and document similarity analysis. </div><br>

<div style="text-align: justify"> MultiLabelBinarizer is used to handle categorical variables before fitting a model, as most machine learning algorithms can only handle numerical data.</div><br>


**2.2 Choose at least 2 ML methods, and perform the model training, with default parameter
configuration, evaluating the performance on both training and test set. Output the confusion
matrix and classification report. Do you observe overfitting or under-fitting? Which model
generates the best performance?**

<br><left><b><font size=4> Random Forest (RF)<b><left>

<div style="text-align: justify">Random Forest (RF) serves as a classification model that constructs a collection of decision trees (DT) using a randomly chosen subset of the given training set. The model aggregates the individual decisions made by each decision tree and combines their votes to make the ultimate prediction.</div>

In [ ]:
model_rf = RandomForestClassifier(n_estimators=100) #(n_estimators=30, max_depth=50)

st = time.time()
# Trainning the model
model_rf.fit(X_train, y_train_mlb)
et = time.time()

# get the execution time
elapsed_time = et - st
print(r'Time to train the model:', elapsed_time,'seconds')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Predictions on training set
y_train_predictions = model_rf.predict(X_train)

# Predictions on test set
y_test_predictions = model_rf.predict(X_test)

In [ ]:
from sklearn import metrics

# Evaluate the model's performance on the test data
accu = model_rf.score(X_train, y_train_mlb)
accuracy = metrics.accuracy_score(y_train_mlb, y_train_predictions)
print(f"Accuracy of the 'Random Forest' model for the trainning set: {accuracy:.2f}, {accu:.2f}")

# Evaluate the model's performance on the test data
accu = model_rf.score(X_test, y_test_mlb)
accuracy = metrics.accuracy_score(y_test_mlb, y_test_predictions)
print(f"Accuracy of the 'Random Forest' model for test set: {accuracy:.2f}, {accu:.2f}")

<br>
<left><b><font size=3 >Classification Report<b><left>

##### Training Set

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Evaluate performance on training set
report_trainning = classification_report(y_train_mlb, y_train_predictions, target_names=mlb.classes_, output_dict=True)
df_report_trainning = pd.DataFrame(report_trainning).transpose()
df_report_trainning

##### Validation Set (Test set)

In [ ]:
# Evaluate performance on test set
report_test = classification_report(y_test_mlb, y_test_predictions, target_names=mlb.classes_, output_dict=True)
df_report_test = pd.DataFrame(report_test).transpose()
df_report_test

<br>
<left><b><font size=3> Confusion Matrix <b><left>

##### Training Set

In [ ]:
confusion_rf_train = multilabel_confusion_matrix(y_train_mlb, y_train_predictions)

# Printing the confusion matrix
for i, label in enumerate(mlb.classes_):
    print(f"Confusion Matrix for '{label}':")
    print(confusion_rf_train[i])
    print()


In [ ]:
# Plotting the confusion matrix
plt.figure(figsize=(15, 8))
for i, label in enumerate(mlb.classes_):
    plt.subplot(3, 3, i + 1)
    plt.title(f"Confusion Matrix for '{label}'")
    plt.imshow(confusion_rf_train[i], cmap='Blues', interpolation='nearest')
    plt.colorbar()
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.xticks(np.arange(2), ['Negative', 'Positive'])
    plt.yticks(np.arange(2), ['Negative', 'Positive'])

plt.tight_layout()
plt.show()

##### Validation Set

In [ ]:
confusion_rf_test = multilabel_confusion_matrix(y_test_mlb, y_test_predictions)

# Printing the confusion matrix
for i, label in enumerate(mlb.classes_):
    print(f"Confusion Matrix for '{label}':")
    print(confusion_rf_test[i])
    print()    

In [ ]:
# Plotting the confusion matrix
plt.figure(figsize=(15, 8))
for i, label in enumerate(mlb.classes_):
    plt.subplot(3, 3, i + 1)
    plt.title(f"Confusion Matrix for '{label}' in Test Set")
    plt.imshow(confusion_rf_test[i], cmap='Oranges', interpolation='nearest')
    plt.colorbar()
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.xticks(np.arange(2), ['Negative', 'Positive'])
    plt.yticks(np.arange(2), ['Negative', 'Positive'])

plt.tight_layout()
plt.show()

<br>
<left><b><font size=4>K-Nearest Neighbors (KNN)<b><left>

<div style="text-align: justify">K-Nearest Neighbors (KNN) operates as a supervised learning classifier that relies on the concept of proximity to perform classifications or predictions for individual data points. Its fundamental principle is grounded in the notion that similar data points tend to cluster together. In the context of classification tasks, KNN assigns a class label to a data point by considering the majority vote of its nearest neighbors. Put simply, it selects the label that is most prevalent among the neighboring data points in close proximity to the one being evaluated.</div><br>

In [ ]:
y_train_mlb
X_train

In [ ]:
# Create the k-NN model
knn = KNeighborsClassifier()

st = time.time()
# Train the model on the training data
knn.fit(X_train, y_train_mlb)
et = time.time()

# get the execution time
elapsed_time = et - st
print(r'Time to train the model:', elapsed_time,'seconds')

In [ ]:
# Generate predictions on the test set
predictions_train = knn.predict(X_train) 

# Generate predictions on the test set
predictions_test = knn.predict(X_test)

In [ ]:
# Evaluate the model's performance on the trainning data
accuracy = knn.score(X_test, y_test_mlb)
print(f"Accuracy of the k-NN model for the trainning set: {accuracy:.2f}")

# Evaluate the model's performance on the test data
accuracy = knn.score(X_test, y_test_mlb)
print(f"Accuracy of the k-NN model for the test set: {accuracy:.2f}")

<br>
<left><b><font size=3 >Classification Report<b><left>


##### Training Set

In [ ]:
# Classification Report for trainning set
report_train_knn = classification_report(y_train_mlb, predictions_train, target_names=mlb.classes_, output_dict=True)
df_report_train = pd.DataFrame(report_train_knn).transpose()
print("Classification Report for Trainning set:")
df_report_train

##### Validation Set

In [ ]:
# Classification Report for test set
report_test_knn = classification_report(y_test_mlb, predictions_test, target_names=mlb.classes_, output_dict=True)
df_report_test_knn = pd.DataFrame(report_test_knn).transpose()
print("Classification Report for Test set:")
df_report_test_knn

<br>
<left><b><font size=3> Confusion Matrix <b><left>

The code presented below prints a series of confusion matrices for each class, displaying True-Positive (top-left), False-Negative (bottom-left), False-Positive (top-right), and True-Negative (bottom-right) counts.
- True Positives (TP): Predicted correctly as positive.
- False Positives (FP): Predicted as positive but actually negative.
- False Negatives (FN): Predicted as negative but actually positive.
- True Negatives (TN): Predicted correctly as negative.

Each value in the confusion matrix represents the count of instances falling into these categories for a specific label.

In [ ]:
# Evaluate with a confusion matrix and classification report
confusion_knn_test = multilabel_confusion_matrix(y_test_mlb, predictions_test)
# Printing the confusion matrix
for i, label in enumerate(mlb.classes_):
    print(f"Confusion Matrix for {label}:")
    print(confusion_knn_test[i])
    print()

In [ ]:
# Plotting the confusion matrix
plt.figure(figsize=(15, 8))
for i, label in enumerate(mlb.classes_):
    plt.subplot(3, 3, i + 1)
    plt.title(f"Confusion Matrix for '{label}'")
    plt.imshow(confusion_knn_test[i], cmap='Oranges', interpolation='nearest')
    plt.colorbar()
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.xticks(np.arange(2), ['Negative', 'Positive'])
    plt.yticks(np.arange(2), ['Negative', 'Positive'])

plt.tight_layout()
plt.show()

<div style="text-align: justify"> To know whether the model is underfitting or overfitting, we must first define these two terms. <br>
<div style="text-align: justify"><br><b>Underfitting</b> occurs when a model is too simple to capture the underlying patterns in the training data, resulting in poor performance even on the training set.  One indicator used to identify this modelling error is to look at the results; if both training and validation/testing performance are poor, the model is considered to be underfitted.</div><br>

<div style="text-align: justify"><b>Overfitting</b> occurs when a model not only learns the underlying patterns in the training data, but also captures noise and random fluctuations, causing it to perform poorly on new and unknown data. One indicator used to identify this modelling error is to look at the results, if the model performs well on training data but poorly on validation or test data, it is considered overfitted.</div><br>

<div style="text-align: justify">Once these two terms have been defined, it is possible to move on to the results obtained for both models. It is important to mention that, for both tests performed with the different classifiers, the default parameters were used.</div><br>
<div style="text-align: justify">For the <b>Random Forest (RF)</b> case, the default parameter implied the number of estimators equal to 100, and the tree depth was set to <i>'None'</i>. On the other hand, for the <b>K-Nearest Neighbor (KNN)</b> classifier, the number of estimators was set to 5, the leaf size to 30 and the type of distance calculation was <i>'Euclidean distance'</i> (p=2), all default parameters of the classifier.</div><br>
    
<b> Random Forest (RF)</b>
<div style="text-align: justify"> In the classification report of the training and test sets, for most of the classes, accuracy, recall and F1 score are slightly lower in the test set compared to the training set. This was expected, as models tend to generalise slightly worse with unseen data. However, the drop in performance is not significant, which indicates that the model still performs well on the test set. Furthermore, the accuracy obtained for the training set was 99%, while 98% for the test set.<br><br>
    
<div style="text-align: justify">As can be seen in the validation set report, the model was not able to correctly classify instances of the <i>'Impact'</i> class, performing very poorly (0%) on all precision, recall and F1 score metrics. This result could be due to the default parameters set to train the model.  Tree depth is one of the most important parameter for tuning the model, as it sets the stop condition that limits the number of splits or levels deep a decision tree can go.</div><br>

<div style="text-align: justify">To enhance the classifier results, it is necessary to adjust the maximum depth of the decision trees when performing hyperparameter fitting for a random forest model. The <i>'weighted avg'</i> metric also showed a decrease in performance on the test set, indicating that the model does not perform as well on the test set across all classes, considering the distribution of classes. Overall, the performance metrics on the test set remain high, indicating that the overfitting is not critical. </div><br>

<b> K-Nearest Neighbor (KNN)</b>

<div style="text-align: justify">On the other hand, the classification report obtained for the KNN classifier showed a small difference between the training set and the validation set, for the test set the values obtained for each of the metrics; accuracy, recall, f1-score, were slightly lower compared to the results obtained for the training set.</div><br>

<div style="text-align: justify"> For the <i>'Impact'</i> class, the metrics derived in the training set were significantly lower compared to the validation set, for all metrics. This improvement in the accuracy, recall and f1-score parameters for the <i>'Impact'</i> class in the test set indicates that the model's predictions for this class are more accurate and reliable when evaluated with new, unseen data.  Producing a recall of 50%, which means, that the model only correctly predicted this class for 50% of the evaluated intents. </div><br>

<div style="text-align: justify">The difference obtained in this class for the training and validation sets suggests that the model did not sufficiently fit the <i>'Impact'</i> class during training, and then after the selection of the nearest neighbour from the test set, the model adjusted its predictions to better capture the features of the <i>'Impact'</i> class. However, it is important to note that the KNN classifier does not fit the data, it does not learn from the model, it only calculates the distance to the nearest points and selects the class according to the majority result of the nearest neighbours.</div><br>

<div style="text-align: justify"> In general, the observed results do not indicate underfitting or overfitting, in fact, the average accuracy obtained in the classification report was 99%, matching with the obtained in the calculated accuracy score (98% for both sets). The high values of the <i>'micro-average'</i> in both sets suggest a good overall performance of the model. While the <i>'macro-average'</i> values are higher only in the test set, indicating that the model performs better after calculating the Euclidean distance of each point.</div><br>


**2.3 Tune the hyper-parameters of the models through cross-validation. How do performance vary?**

#### K-Fold Cross-Validation

For this experiment, a value of 5 K-fold cross-validation was selected.

In [ ]:
from sklearn.model_selection import cross_val_score,KFold
from sklearn.metrics import make_scorer, f1_score, precision_score

scorer = make_scorer(f1_score, average='macro')
kf = KFold(n_splits=5)
kf.get_n_splits(X_train)

for i, (train_index,test_index) in enumerate(kf.split(X_train)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")

##### Random Forest 

In [ ]:
cross_val_score(model_rf, X_train, y_train_mlb, cv=kf, scoring = scorer)

##### K-Nearest Neighbor

In [ ]:
cross_val_score(knn, X_train, y_train_mlb, cv=kf, scoring=scorer)

#### Grid Search Technique

<div style="text-align: justify">Hyperparameters are settings that control the learning process of machine learning models. While the parameters are learned during the training process, the hyperparameters are set before the training starts. Therefore, in order to find the parameters that best fit the performance of the model, the GridSearch technique was applied. This technique applies all possible combinations of hyperparameters, resulting in a set of parameters that will improve the performance of the model.</div>

#### Random Forest

In [ ]:
#GRID SEARCH
from sklearn.model_selection import GridSearchCV

# Create the parameter grid based on the results of random search 
params = {'n_estimators': [10, 50, 80], 'max_depth': [15, 25, 50] } #'criterion' :['gini', 'entropy']}

# Instantiate the grid search model
gs_rf = GridSearchCV(model_rf, param_grid = params, scoring='f1_macro', cv = 5, verbose = 1) 
# scoring='accuracy'
# cv: that's the number of fold for the cross-validation
# verbose: specifies the verbosity level of the GridSearchCV object. 

# Trainning the model
st = time.time()
gs_rf.fit(X_train, y_train_mlb)
et = time.time()

# get the execution time
elapsed_time = et - st
print(r'Time used by Grid Search:', elapsed_time,'seconds')

In [ ]:
best_params_rf = gs_rf.best_params_
print(r'The best combination of parameters the Grid Search has found is:', best_params_rf)
print("Best F1-Score: {:.2f}".format(gs_rf.best_score_))

#### Ask to the prof which graph we should use -------------------------------------------------------------------------

In [ ]:
# Bar plot

In [ ]:
# Let's print the f1 macro reached for each combination
y = gs_rf.cv_results_["mean_test_score"].tolist()
x = [i for i in range (1, len(y)+1)]
mean_test_score_df = pd.DataFrame()
mean_test_score_df["f1_macro"] = y

plt.figure(figsize=(8, 6))
sns.barplot(x = mean_test_score_df.index, y = "f1_macro", data = mean_test_score_df, color='blue')

# Add a title and labels to the plot
plt.title('F1-macro Scores for Different Parameters')
plt.xlabel('Combination')
plt.ylabel('F1-macro Score')
plt.show()

In [ ]:
# Heatmap plot

In [ ]:
results_rf = pd.DataFrame(gs_rf.cv_results_)
results_rf

In [ ]:
# We create a pivot table 
scores_rf = results_rf.pivot(index='param_max_depth', columns='param_n_estimators', values='mean_test_score')
scores_rf

In [ ]:
sns.heatmap(scores_rf, annot=True, cmap='viridis', fmt='.5g')
plt.xlabel('param_max_depth')
plt.ylabel('param_n_estimators')
plt.title('Mean F1-Score over all folds for each combination of parameters')
plt.show()

#### K-Nearest Neighbor

In [ ]:
#GRID SEARCH tuning
from sklearn.model_selection import GridSearchCV

params = {'n_neighbors': [5, 7, 9], 'leaf_size': [5, 15, 20]} #'metric': ['euclidean', 'manhattan']} 
grid_search_knn = GridSearchCV(knn, params, scoring='f1_macro', cv = 5, verbose=1)
# scoring = 'accuracy'

st = time.time()
grid_search_knn.fit(X_train, y_train_mlb)
et = time.time()

# get the execution time
elapsed_time = et - st
print(r'Time used by Grid Search:', elapsed_time,'seconds')

In [ ]:
best_params_knn = grid_search_knn.best_params_
print(r'The best combination of parameters the Grid Search has found is:', best_params_knn)
print("Best F1-Score: {:.2f}".format(grid_search_knn.best_score_))

#### Aks to the prof which graph we should use --------------------------------------------------------------------------

In [ ]:
#Bar plot

In [ ]:
# Let's print the f1 macro reached for each combination
y = grid_search_knn.cv_results_["mean_test_score"].tolist()
x = [i for i in range (1, len(y)+1)]
mean_test_score_df = pd.DataFrame()
mean_test_score_df["f1_macro"] = y
#print(mean_test_score_df)

sns.barplot(x = mean_test_score_df.index, y = "f1_macro", data = mean_test_score_df, color='blue')

# Add a title and labels to the plot
plt.title('F1-macro Scores for Different Parameters')
plt.xlabel('Combination')
plt.ylabel('F1-macro Score')

# Display the plot
plt.show()

In [ ]:
# Heatmap plot

In [ ]:
results_knn = pd.DataFrame(grid_search_knn.cv_results_)
results_knn

In [ ]:
# We create a pivot table before create the heatmap
scores_knn = results_knn.pivot(index='param_leaf_size', columns='param_n_neighbors', values='mean_test_score')
scores_knn

In [ ]:
sns.heatmap(scores_knn, annot=True, cmap='viridis', fmt='.5g')
plt.xlabel('param_n_neighbors')
plt.ylabel('param_leaf_size')
plt.title('Mean F1-score over all folds for each combination of parameters')
plt.show()

**2.4. Comments on the results for each on the intents.**

<div style="text-align: justify">The aim of this analysis is to assess the predictive capability of two models in classifying attack labels. The models will be evaluated based on the hyperparameters identified previously.</div>

##### Evaluation of Random Forest with tuned parameters

In [ ]:
# Random Forest with the hyperparameters
model_rf_tunned = RandomForestClassifier(n_estimators = 25, max_depth = 10) # (n_estimators=30, max_depth=50)

st = time.time()
# Trainning the model
model_rf_tunned.fit(X_train, y_train_mlb)
et = time.time()
# get the execution time
elapsed_time = et - st
print(f'Time to train the model:', elapsed_time,'seconds','\n')

# Predictions on test set
y_test_pred_tune = model_rf.predict(X_test)

# Evaluate the model's performance on the test data
accuracy = metrics.accuracy_score(y_test_mlb, y_test_pred_tune)
print(f"Accuracy of the 'Random Forest' model for test set: {accuracy:.2f}",'\n')

# Evaluate performance on test set
report_test_tune = classification_report(y_test_mlb, y_test_pred_tune, target_names=mlb.classes_, output_dict=True)
df_report_test_tune = pd.DataFrame(report_test_tune).transpose()
print(f'         Classification Report Trainning Set', '\n')
print(df_report_test_tune)

In [ ]:
# Compute the heatmap of the correlation matrix
plt.figure(figsize=(5, 3))
sns.heatmap(df_report_test_tune.loc["Defense Evasion" : "Persistence"], cmap='Blues', annot=True, vmin=.0, vmax=1,fmt='.3f')
plt.xlabel('Intents')
plt.ylabel('Evaluation technique')
plt.title('Intents classification report')
plt.show()

##### Evalutaing of K-Nearest Neighbors with tuned parameters 

In [ ]:
# Create the k-NN model
knn_tune = KNeighborsClassifier(leaf_size=5, n_neighbors=5)

# Train the model on the training data
t = time.time()
knn_tune.fit(X_train, y_train_mlb)
et = time.time()

# get the execution time
elapsed_time = et - st
print(r'Time to train the model:', elapsed_time,'seconds','\n')

# Generate predictions on the test set
predictions_knn_tune = knn.predict(X_test)

# Evaluate the model's performance on the test data
accuracy_knn_tune = knn.score(X_test, y_test_mlb)
print(f"Accuracy of the k-NN model: {accuracy_knn_tune:.2f}",'\n')

report_knn_tune = classification_report(y_test_mlb, predictions_knn_tune, target_names = mlb.classes_, output_dict=True)
df_report_knn_tune = pd.DataFrame(report_knn_tune).transpose()
print("              Classification Report for KNN",'\n')
print(df_report_knn_tune)

In [ ]:
# Compute the heatmap of the correlation matrix
plt.figure(figsize=(5, 3))
sns.heatmap(df_report_knn_tune.loc["Defense Evasion" : "Persistence"], cmap='Blues', annot=True, vmin=.0, vmax=1,fmt='.3f')
plt.xlabel('Intents')
plt.ylabel('Evaluation technique')
plt.title('Classification report')
plt.show()

Need to write how the hyperparameter tunning improves the result; especially in the intent Impact, for both models... I'll do it later.

**2.5 Explore the possible features: try combining features differently, e.g., does tf-idf improve or worsen performance? Think about the problem and summarize the ways you have tried (even those that did not work).**

<div style="text-align: justify"><b>First Attempt</b></div><br>

<div style="text-align: justify">For the first attempt, 33 features were used for the training set. The classifiers selected to perform the predictions were 'Random Forest' and 'K-Nearest Neighbor' with both models using the default parameters. In the performance evaluation, an accuracy of 98% was obtained for both models in the validation set.  While in the training set it reached 99% for RF and 98% for KNN. The decrease in accuracy achieved by RF in the validation set suggests a slight overfitting, however, it is a tolerable value that assumes that the model still performs well on the test set.</div><br>

<div style="text-align: justify">According to the classification report, for almost all attempts the values of precision, recall and f1-scores reached high percentages, around 98.9%.  Except for the 'Impact' and 'Harmless' intents when applying RF as a model, the results obtained for these classes were quite lower compared to those obtained for the other classes, both in the training set and in the validation set, where 0% was obtained in each metric. The model presented a very poor performance when trying to classify these two classes.</div><br>

<div style="text-align: justify">As for tuning techniques, K-fold cross-correlation and Grid Search were applied to see if the results obtained previously could be improved. However, when looking at the results obtained with 5 folds, the performance decreased considerably reaching an average of 77% accuracy. Suggests that the model may be in overfitting. </div><br>

<div style="text-align: justify">Therefore, for the next attempt we consider a reduction of the dimensionality of the data set to improve generalization.</div><br>

<div style="text-align: justify"><b>Second attempt</b></div><br>
<div style="text-align: justify">For the second attempt, the number of features was reduced to only 12 for the training set.  The classifiers used in the previous attempt, RF and KNN, were kept for this evaluation with both using their default parameters. The results obtained in the performance evaluation, the accuracy did not change from the previous attempt, reaching 98% for the RF case and 98% for the KNN for both the training and validation sets, indicating that the models continue to make good predictions for the classes.<br>
<br>
<div style="text-align: justify">If we take a look at the classification report for almost all attempts, the values for accuracy, recall and f1-scores reached high percentages, around 98%. Nevertheless, the performance of the 'Impact' class was very poor in both models for the classification obtained in the validation data, with 0% for every metric (precision, recall and f1-score), indicating that the number of features selected to train both models was not sufficient to be able to correctly classify this class.<br><br>
<div style="text-align: justify">In both training and validation reports, the F1 score obtained for the classes "Impact" and "Harmless" was very low compared to the other classes. It seems that for both models, these two classes are the most difficult to classify correctly. This could be due to the fact that, the number of data selected during the splitting of the training set did not cover enough samples of these two classes, as they are the least sampled attempts of the whole dataset. </div><br>

<div style="text-align: justify">After conducting multiple tests and adjusting the number of features in the training set, the results showed similar levels of accuracy. However, upon further analysis of the classification report parameters, it was found that the highest metrics for accuracy, recall, and F1-score were achieved with a number of features greater than 22. The F1-score parameter indicated that the models performed better in classifying samples belonging to the 'Impact' and 'Harmless' classes, which were more difficult to detect in almost all the tests. </div><br>

### <left><b><font size=4>Section 3 – Unsupervised Learning – Clustering<b><left>

<div style="text-align: justify">Cluster the attacks according to their characteristics. Choose at least 2 Clustering Algorithms, and for each of them solve the following points.</div>

In [ ]:
df_exploded = result_df_corr.explode("Set_Fingerprint").reset_index()
df_exploded = df_exploded.drop(columns=['index'])
#features_exploted = result_df_exploded.drop(columns=["session_id", "full_session", "first_timestamp", "Set_Fingerprint", "bag_of_words", "tfid"])

# We should use the stardardized data (X and y features)
labels = df_exploded["Set_Fingerprint"]
result_df_exploded = df_exploded.filter(features_names)
result_df_exploded

<div style="text-align: justify"> Two clustering algorithms will be implemented to our dataset, the chosen were MiniBatchK-Means and Gaussian Mixture Model (GMM). <br>
First of all we have to select the features to be evaluate, <b>'Pricipal Component Analysis (PCA)'</b> was used to reduce the dimensionality of the data.</div>

#### Dimensionality Reduction by applying 'PCA'

In [ ]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA

# Reduce dimensionality using PCA----------------------------------------
pca = PCA(n_components=10)        # Adjust the number of components as needed
reduced_data = pca.fit_transform(result_df_exploded)

#### K-Means

In [ ]:
# k-Means with n_clusters = 3
# cl_labels3 = kmeans.fit_predict(features_exploted) # Get clusters ID
kmeans = KMeans(n_clusters=3)
cl_labels3 = kmeans.fit(reduced_data) # Using the features selected

# print the clustered labels
print('The clustered labels are:\n', kmeans.labels_)
print()

# print the centroid of each feature for each cluster
print('The centroids are:\n', kmeans.cluster_centers_)
print()

In [ ]:
# Unsupervised metric  (it takes approx 2500 sec o 3000 sec)
silhouette  = silhouette_score(reduced_data, kmeans.labels_)

# Supervised metrics
ri = rand_score(np.ravel(labels), kmeans.labels_)
ari = adjusted_rand_score(np.ravel(labels), kmeans.labels_)

In [ ]:
print('k-Means with 3 clusters')
(unique, counts)=np.unique(kmeans.labels_, return_counts=True)
print("Size of each cluster: ", counts)
print(f'k_means clustering error: {round(kmeans.inertia_, 2)}')
print(f'Silhouette: {round(silhouette, 2)}')
print(f'RI: {round(ri, 2)}')
print(f'ARI: {round(ari, 2)}')

#### Gaussian mixture model (GMM)

In [ ]:
gmm = GaussianMixture(n_components=3)
gmm.fit(result_df_exploded)           # Get clusters ID

In [ ]:
# print the clustered labels
gmm_labels = gmm.predict(result_df_exploded)
print('The clustered labels are:\n', gmm_labels)

In [ ]:
# Unsupervised metric
silhouette  = silhouette_score(result_df_exploded, gmm_labels)
log_l = gmm.score(result_df_exploded)

# Supervised metrics
ri = rand_score(np.ravel(labels), gmm_labels)
ari = adjusted_rand_score(np.ravel(labels), gmm_labels)

In [ ]:
# report effective size
print("Effetive size of each cluster: ", gmm.weights_)
# report usupervised and supervised metric
print(f'GMM total log-likelihood score: {round(log_l, 2)}')
print(f'Silhouette: {round(silhouette, 2)}')
print(f'RI: {round(ri, 2)}')
print(f'ARI: {round(ari, 2)}')

**3.1. Determine the number of clusters: This can be done using methods like the elbow method or
silhouette analysis. Explain your reasoning.**

<div style="text-align: justify"> The clustering error is strongly dependent form the number of cluster. 
The best situation, with the lowest error possible, is obiuously when we have the same number of clusters of the datapoins. Actually in this situation we're not even performing clustering, but the clustering error will be zero.<br>
<br>
Now we want to find the number of clusters that gives us the best results, so with the lowest clustering error. 
We'll follow 2 approches for each clustering algorithm applied, elbow method and validation error. </div>

#### K-Means

In [ ]:
# Step 1: Elbow Method  (260 sec)
n_cluster_list=[]
inertia_list=[]
shs_list = []
ri_list = []
ari_list = []

for n_clusters in range(3, 16):
    kmeans_new = KMeans(n_clusters=n_clusters)
    cl_labels = kmeans_new.fit_predict(reduced_data)
    
    # Unsupervised metric  (If we evaluate the silhoutte it takes too long)
    # silhouette = silhouette_score(reduced_data, cl_labels)
    # n_cluster_list.append(n_clusters)
    # shs_list.append(silhouette)
    
   # Rand Index and Adjusted Rand Index:
    ri_list.append(rand_score(np.ravel(labels), cl_labels))
    ari_list.append(adjusted_rand_score(np.ravel(labels), cl_labels))
    
    # For Elbow Method (Inertia):
    inertia_list.append(kmeans_new.inertia_)
    n_cluster_list.append(n_clusters)

**Elbow Method**

In [ ]:
# Step 3
# Plot k-Means clustering error 
# Set up Seaborn style
sns.set(style="darkgrid")

plt.figure(figsize=(5, 3.5))
plt.plot(n_cluster_list, inertia_list, marker='o', markersize=5, color='blue')
plt.xlabel('Number of clusters')
plt.ylabel('k-Means clustering error')
plt.title('k-Means Clustering Error Analysis')
plt.show()

In [ ]:
# Plot RI
# Set up Seaborn style
sns.set(style="darkgrid")

plt.figure(figsize=(5, 3.5))
plt.plot(n_cluster_list, ri_list, marker='o', markersize=5, color='blue')
plt.xlabel('Number of clusters')
plt.ylabel('RI')
plt.title('Rand Index Analysis')
plt.show()

In [ ]:
# Plot ARI
# Set up Seaborn style
sns.set(style="darkgrid")

plt.figure(figsize=(5, 3.5))
plt.plot(n_cluster_list, ari_list, marker='o', markersize=5, color='blue')
plt.xlabel('Number of clusters')
plt.ylabel('ARI')
plt.title('Adjusted Rand Index')
plt.show()

**THE CODE BELOW: IT's used if we would have chosen to perform the silhouette analysis, but it is not mandatory, of course we have to justify why we do not choose this one, computational time and so on...**

In [ ]:
# Silhouette
# Get n_clusters leading to the highest silhouette
best_sh = np.max(shs_list)
best_n = n_cluster_list[np.argmax(shs_list)]
print("best k: ",best_n, " with corresponding silhouette: ", best_sh)

# Plot
plt.figure(figsize=(5, 3.5))
plt.plot(n_cluster_list,shs_list, marker='o', markersize=5)
plt.scatter(best_n, best_sh, color='r', marker='x', s=90)
plt.grid()
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette score')
plt.show()

#### Gaussian Mixture Model (GMM)

In [ ]:
n_cluster_list=[]
shs_list = []
ri_list = []
ari_list = []
log_l_list=[]

for n_clusters in range(3, 16):
    gmm = GaussianMixture(n_components=n_clusters)
    cl_labels = gmm.fit_predict(X_s)
    
    # silhouette  = silhouette_score(X_s, cl_labels)
    # n_cluster_list.append(n_clusters)
    # shs_list.append(silhouette)
    
    ri_list.append(rand_score(np.ravel(y), cl_labels))
    ari_list.append(adjusted_rand_score(np.ravel(y), cl_labels))
    log_l_list.append(gmm.score(X_s))

**Elbow Method**

In [ ]:
#3. Has the performance improved also on the other metrics? Plot the other metrics for the different values of n_cluster.
# Set up Seaborn style
sns.set(style="darkgrid")

# Plot GMM total log-likelihood score
plt.figure(figsize=(5, 3.5))
plt.plot(n_cluster_list,log_l_list, marker='o', markersize=5)
plt.xlabel('Number of clusters')
plt.ylabel('GMM total log-likelihood score')
plt.show()

# Plot ARI
plt.figure(figsize=(5, 3.5))
plt.plot(n_cluster_list,ari_list, marker='o', markersize=5)
plt.xlabel('Number of clusters')
plt.ylabel('ARI')
plt.show()

In [ ]:
#2. Plot the silhouette score for the different values of n_cluster. 
# What is the best value of k, leading to the highest Silhouette.

# Get n_clusters leading to the highest silhouette
best_sh= np.max(shs_list)
best_n=n_cluster_list[np.argmax(shs_list)]
print("best k: ",best_n, " with corresponding silhouette: ", best_sh)

# Plot
# Set up Seaborn style
sns.set(style="darkgrid")
plt.figure(figsize=(5, 3.5))
plt.plot(n_cluster_list,shs_list, marker='o', markersize=5)
plt.scatter(best_n, best_sh, color='r', marker='x', s=90)
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette score')
plt.show()

**3.2. Tune other hyper-parameters, if any.**

##### Tunning the hyperparameter of K-Means

In [ ]:
# Step 1: Elbow Method
n_cluster_list=[]
inertia_list=[]
shs_list = []
ri_list = []
ari_list = []

for n_clusters in range(3, 16):
    kmeans_tunning = KMeans(n_clusters=n_clusters, init='k-means++', random_state=None, n_init=1)
    labels_tunning = kmeans_tunning.fit_predict(reduced_data) # Using the features selected
    # kmeans_new = KMeans(n_clusters=n_clusters)
    # cl_labels = kmeans_new.fit_predict(reduced_data)
    
    # Rand Index and Adjusted Rand Index:
    ri_list.append(rand_score(np.ravel(labels), labels_tunning))
    ari_list.append(adjusted_rand_score(np.ravel(labels), labels_tunning))
    
    # For Elbow Method (Inertia):
    inertia_list.append(kmeans_tunning.inertia_)
    n_cluster_list.append(n_clusters)

In [ ]:
# Step 3
# Plot k-Means clustering error 
# Set up Seaborn style
sns.set(style="darkgrid")
plt.figure(figsize=(5, 3.5))
plt.plot(n_cluster_list, inertia_list, marker='o', markersize=5, color='orange')
plt.xlabel('Number of clusters')
plt.ylabel('k-Means clustering error')
#plt.title('k-Means Clustering Error')
plt.show()

In [ ]:
# Plot RI
# Set up Seaborn style
sns.set(style="darkgrid")

plt.figure(figsize=(5, 3.5))
plt.plot(n_cluster_list, ri_list[:13], marker='o', markersize=5, color='orange')
plt.xlabel('Number of clusters')
plt.ylabel('RI')
plt.title('Rand Index')
plt.show()

In [ ]:
# Plot ARI
# Set up Seaborn style
sns.set(style="darkgrid")

plt.figure(figsize=(5, 3.5))
plt.plot(n_cluster_list, ari_list, marker='o', markersize=5, color='orange')
plt.xlabel('Number of clusters')
plt.ylabel('ARI')
plt.title('Adjusted Rand Index')
plt.show()

**3.3. Visualize the clusters through t-SNE visualization.**

Let's apply the k-mean with the best hyper-parameters found:

In [ ]:
kmeans = KMeans(n_clusters=15, init='k-means++', random_state=None, n_init=1)
cl_labels3 = kmeans.fit_predict(reduced_data) # Using the features selected

# print the clustered labels
print('The clustered labels are:\n', kmeans.labels_)
print()

# print the centroid of each feature for each cluster
print('The centroids are:\n', kmeans.cluster_centers_)
print()

The goal of t-SNE is to take a set of points in a high-dimensional space and find a faithful representation of those points in a lower-dimensional space, typically the 2D plane.

In [ ]:
labels_df = pd.DataFrame(reduced_data)
labels_df['cluster'] = cl_labels3
labels_df

In [ ]:
X = labels_df.iloc[:,:-1]
print(X)
tsne = TSNE(n_components=2).fit_transform(X)

df_tsne = pd.DataFrame(tsne)
df_tsne["cluster"] = cl_labels3
df_tsne.columns = ["x1", "x2", "cluster"]
df_tsne



In [ ]:
fig, ax = plt.subplots(figsize=(12,6))
sns.scatterplot(data=df_tsne, x='x1', y='x2', hue='cluster', legend="full", alpha=0.5)
ax.set_title('2D TSNE cluster visualization')


ax.set_xlim([-200, 300])  
ax.set_ylim([-220, 260])  

**3.4. Cluster analysis. Analyze the characteristics of each cluster. This might involve examining the most frequent words in each cluster (try word cloud). Try to understand which are the most
representative.**

**3.5. Do clusters reflect intent division, i.e., are the clusters homogeneous in terms of intents? How are intents divided into the clusters?**

**3.6. Find clusters of similar attacks, study their sessions and try to associate with them specific categories of attacks (more fine grained than the ones of MITRE ATT&CK Tactics). As an example, see the image below, where we perform a similar exercise (through graph community detections). NOTE: you do not have to do this exercise for all the clusters, but only on some examples.**

<br><left><b><font size=4>Section 4 – Language Models exploration<b><left>

<div style="text-align: justify">Experiment language models for solving the same supervised task as in Section 2. In this task, the objective is to harness the capabilities of language models like Bert or Word2Vec, for supervised learning (assign intents to sessions). 
<br><br> Two interesting concepts play a role when we use neural networks:
<div style="text-align: justify"><br><b>1)</b> It is possible to do transfer learning, i.e., to take a model that have been trained with other enormous datasets by Big Tech companies, and we can do fine-tuning i.e., to train this model starting from its pre-trained version.
<br><b>2)</b> In NLP tasks, words/documents are transformed into vectors (encoding) and this task is Unsupervised, so we can use a much larger amount of data.
</div>

<div style="text-align: justify"><b>4.1. If you choose Doc2Vec: pretrain Doc2Vec on body column of the session text. If you chose Bert: take the pretrained Bert model like in this example. (NB: In this tutorial they used BertForSequenceClassification, but if you want to continue with step 2, you must take an other Bert implementation from HuggingFace)</div></b>

##### Data Preparation

In [ ]:
df_original = pd.read_parquet('ssh_attacks.parquet')
data = df_original.copy()
data

In [ ]:
def clean_text(text):
    # Filter out non-alphabetic characters and convert to lowercase
    cleaned_text = ' '.join(word.lower() for word in text.split() if word.isalpha())
    return cleaned_text

# Apply text cleaning to the 'full_session' column
data['cleaned_session'] = data['full_session'].apply(clean_text)

In [ ]:
# Tokenize 'cleaned_session' texts and create tagged documents
tagged_data = [TaggedDocument(words=session.split(), tags=[str(i)]) for i, session in enumerate(data['cleaned_session'])]

##### Building and Training Doc2Vec Model

<div style="text-align: justify">Cleaning the text data by removing non-alphabetic characters and converting text to lowercase. The Gensim library was used to train a Doc2Vec model on the cleaned text data. A vocabulary was buildt and the Doc2Vec model was trained to generate the vector of embeddings for each session text.</div>

In [ ]:
# Initialize the Doc2Vec model
doc2vec_model = Doc2Vec(vector_size=100, window=5, min_count=1, epochs=20, dm=1)

# Build vocabulary
doc2vec_model.build_vocab(tagged_data)

# Train the model
doc2vec_model.train(tagged_data, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)

##### Saving the Trained Model

In [ ]:
# Save the trained model
doc2vec_model.save('trained_doc2vec_model.model')

**4.2. Add a last Dense Layer**

<div style="text-align: justify">We have trained the Doc2Vec model, which generated embeddings for our text data. Now, to perform classification, we will build a simple Neural Network that will take these embeddings and will add a dense layer for the classification task. The dense layer will have as many neurons as the number of classes we want to predict.</div>

Created a neural network model using TensorFlow/Keras. Set up the architecture by adding a Dense layer with the input dimension being the size of the Doc2Vec vectors and a final Dense layer with softmax activation for multi-class classification.

In [ ]:
# Initialize a sequential model
model = Sequential()

# Add the Doc2Vec embeddings as the input layer
input_dim = doc2vec_model.vector_size
model.add(Dense(input_dim, input_shape=(input_dim,), activation='relu'))

In [ ]:
# Add the final dense layer for classification
num_classes = 7
model.add(Dense(num_classes, activation='softmax'))  # Softmax activation for multi-class classification
#This code adds the final dense layer for classification on top of the Doc2Vec embeddings. num_classes represents the number of output classes we have in our classification task (in our case, the 7 different intents). The softmax activation function is used here as it's suitable for multi-class classification tasks, providing probabilities for each class.
#The purpose of this code is to create a neural network architecture suitable for classification using the Doc2Vec embeddings as input features. 

<div style="text-align: justify">To compile the model, we'll need to set a few parameters:<br>

- Optimizer: The optimizer adjusts the weights during training to minimize the loss function.
- Loss Function: For multi-class classification, 'categorical_crossentropy' is commonly used.
- Metrics: These are used to judge the performance of the model. For classification tasks, 'accuracy' is a standard metric.</div>

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#Categorical Crossentropy: This loss function is suitable for multi-class classification problems. It measures the dissimilarity between the true distribution and the predicted distribution of the classes.
#Accuracy: It calculates the accuracy of the model, i.e., the number of correctly predicted instances divided by the total number of instances.

**4.3. Fine-tune the last layer of the network on the supervised training set for N epochs.**

<div style="text-align: justify">Once the dataset was divided into training and validation sets. Converted the data into TensorFlow tensors. Fit the neural network model using the training set, specifying the number of epochs and batch size.</div>

In [ ]:
data

In [ ]:
# Load the trained Doc2Vec model
doc2vec_model = Doc2Vec.load('trained_doc2vec_model.model') 

# Function to infer vectors for each document
def infer_vector(text):
    return doc2vec_model.infer_vector(text.split())

# Apply inference to each text and store the vectors in a new column
data['doc2vec_vectors'] = data['cleaned_session'].apply(infer_vector)

# Extract vectors and convert to array for modeling
X = np.array(data['doc2vec_vectors'].tolist())

In [ ]:
data['doc2vec_vectors']

In [ ]:
X = data['doc2vec_vectors']  
y = data['Set_Fingerprint'] 

# Split the data into training and test sets (70% train, 30% test)
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.3, random_state=42)

# Print the shapes 
print(f"X_train shape: {X_train1.shape}, y_train shape: {y_train1.shape}")
print(f"X_test shape: {X_test1.shape}, y_test shape: {y_test1.shape}")

In [ ]:
mlb = MultiLabelBinarizer()
y_train1_mlb = mlb.fit_transform(y_train1)
y_test1_mlb = mlb.transform(y_test1)

In [ ]:
import tensorflow as tf
X_train_array = np.array(X_train1.tolist())
X_test_array = np.array(X_test1.tolist())

X_train_tensor = tf.convert_to_tensor(X_train_array)
X_test_tensor = tf.convert_to_tensor(X_test_array)
y_train_tensor = tf.convert_to_tensor(y_train1_mlb)
y_test_tensor = tf.convert_to_tensor(y_test1_mlb)

In [ ]:
type(y_train1_mlb)

In [ ]:
from tensorflow.keras.models import clone_model
cloned_model = clone_model(model)
cloned_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
history = cloned_model.fit(X_train_tensor, y_train_tensor, epochs=10, batch_size=32, validation_data=(X_test_tensor, y_test_tensor))
#Epoch: An epoch represents one complete pass through the entire training dataset.
#Number of Epochs: It determines the number of times the learning algorithm will work through the entire training dataset
#Batch: The training data is divided into batches. The model is trained on each batch, and the weights are updated after each batch.
#Batch Size: It's the number of samples processed before the model is updated. Smaller batch sizes offer faster training but may be less stable than larger batch sizes.
#model.fit(): This function fits the model on the training data. It iterates over a fixed number of epochs and updates the model's weights based on the backpropagation of errors.

<div style="text-align: justify">When using <i>'tf.function'</i>, TensorFlow creates a graph representation of the computation for optimization. However, certain operations, especially those involving variable creation, can cause issues within the graph when the function is called multiple times. <br><br>
Creating a new model instance or cloning the model before train it again helps to avoid conflicts caused by the pre-existing variables or the TensorFlow graph. This approach prevents potential issues that might arise from reusing the same model object within a <i>'tf.function'</i> context. Essentially, it ensures that the new training session starts with a fresh model instance, avoiding any residual state from previous training sessions that might interfere with the current one.</div><br>

**4.4 Plot the learning curves on training and validation set. After how many epochs should we stop the training?**

<div style="text-align: justify">The Matplotlib library was used to create visualisations of training and validation losses, as well as training and validation accuracy over epochs. These plots are helpful to understand how the model learns over time, showing convergence and possible overfitting or underfitting.</div><br>

<div style="text-align: justify">(Learning curves are used to understand the changes in 'loss' and 'accuracy' at each epoch. If the training accuracy is high but the validation accuracy is low, it may indicate overfitting, whereas consistently low accuracy for both may suggest underfitting).</div><br>

<div style="text-align: justify">[In summary, the process involved preparing the text data, training a Doc2Vec model to generate embeddings, constructing and training a neural network model for intent classification, and visualizing the model's learning progress. The aim is to build a model that effectively identifies attack session intents using text data and Doc2Vec embeddings.]</div>

In [ ]:
# Access the training history
training_loss = history.history['loss']
validation_loss = history.history['val_loss']
training_accuracy = history.history['accuracy']
validation_accuracy = history.history['val_accuracy']

# Set up Seaborn style
sns.set(style="darkgrid")

# Plot training and validation loss
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(training_loss, label='Training Loss')
plt.plot(validation_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot training and validation accuracy
plt.subplot(1, 2, 2)
plt.plot(training_accuracy, label='Training Accuracy')
plt.plot(validation_accuracy, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

<left><b><font size=4>Final Considerations<b><left>